# Advanced Modeling and Simulation Course

### Instructor: Dr. M. Nedim ALPDEMİR

*This material is prepared as part of the EE592-Advanced Modeling And Simulation course given at Yıldırım Beyazıd University (YBÜ).*

## Combined Simulation Example

### Problem Description

![workshop](Workshop.png)

* In a shop floor items arrive at an entry point where they are transferred to a different section for processing. 
* The transfer is performed by an everhead crane. 
* The crane has a dynamic controller that adjusts the transfer process according to the weight of the item.
* The mean interarrival interval of the items is 0.5 minutes and the arrival process is modelled by an exponential distribution
* The mass of the items range between 5 and 17 kg. and is distributed uniformly
* The crane acts as a single service point. Items line in a single queue (on a conveyor) if the crane is not able to cope with the arrival rate.
* The mass of the crane's cart is 10 kg.

![workshop-devs](Workshop-DEVS.png)

* The crane's operation must be  modeled by a set of differential equations. The solution of the differential equations should be done in such a way that the transfer of the item is optimized in terms of smoothness and time.

* The overhead crane is described by the following dynamic equations:

<p class='vspace'>$$\begin{bmatrix} \dot y \\ \dot v \\ \dot \theta \\ \dot q \end{bmatrix}=\begin{bmatrix} 0 && 1 && 0 && 0\\ 0 && 0 && \epsilon && 0\\ 0 && 0 && 0 && 1\\ 0 && 0 && -1 && 0 \end{bmatrix} \begin{bmatrix} y \\ v \\ \theta \\ q \end{bmatrix} + \begin{bmatrix} 0 \\ 1 \\ 0 \\ -1 \end{bmatrix} u$$
</p>

* where $m_1=10$ is the mass of the cart, $m_2=1$ is the mass of the item carried, $\epsilon$ is $m_2/(m_1+m_2)$, $y$ is the position of the overhead cart, $v$ is the velocity of the overhead cart, $\theta$ is the angle of the pendulum relative to the cart, and $q$ is the rate of angle change.


![pendulum Crane](pendulum_crane.png)

<p class='vspace'>The objective of the controller is to adjust the force on the cart to move the  mass to a new final position. Ensure that initial and final velocities and angles of the pendulum are zero. The position of the pendulum mass is initially at the left side of the crane operating range and it is desired to move it to the right side of the operating range within 6.2 seconds. Demonstrate controller performance with changes in the pendulum position and that the final pendulum mass remains at the final position without oscillation.


In [ ]:
"""
@author Dr. M. Nedim ALPDEMİR
"""
from pypdevs.DEVS import AtomicDEVS
import random as rnd

"""
In a shop floor items arrive at an entry point where they are transferred to a different section for preocessing. 
The transfer is performed by an everhead crane. The crane has a dynamic controller that adjusts the transfer
process according to the weight of the item.

The crane acts as a single service point. Items are put in a single queue if the crane is not able to cope with the arrival rate.
operation of the crane must be  modeled by a set of differential equations. The solution of the differential equations should be done 
in such a way that the transfer of the item is optimized in terms of smoothness and time.


"""
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)


class LoadItem:
    def __init__(self, id, min, max, crTime):
        self.id = id
        self.creationTimeInterval = crTime
        self.mass = rnd.uniform(min, max)

        
# Define the state of the generator as a structured object
class GeneratorState:
    def __init__(self):
        # Current simulation time (statistics)
        self.current_time = 0.0
        # Remaining time until generation of new event
        self.remaining = 0.0
        self.avg_delta_arival = 0.0
        self.gen_num = 0

class ItemGenerator(AtomicDEVS):
    def __init__(self, gen_param, min_mass, max_mass):
        AtomicDEVS.__init__(self, "ItemGenerator")
        # Output port for the event
        self.item_arrival = self.addOutPort("out_item_event")
        # Define the state
        self.state = GeneratorState()
        self.min_mass = min_mass
        self.max_mass = max_mass
        # Parameters defining the generator's behaviour. i.e. interarrival rate
        self.gen_param = gen_param
        self.state.remaining = 0
        
    def intTransition(self):
        # Update simulation time
        self.state.current_time += self.timeAdvance()     
        self.state.remaining = rnd.expovariate(self.gen_param)
        self.state.avg_delta_arival = self.state.current_time / self.state.gen_num
        return self.state

    def timeAdvance(self):
        # Return remaining time
        return self.state.remaining

    def outputFnc(self):
        creation_time_interval = self.state.remaining
        self.state.gen_num += 1
        new_item = LoadItem(self.state.gen_num, self.min_mass, self.max_mass, creation_time_interval)
        # Output the new event on the output port
        return {self.item_arrival: new_item}


In [ ]:
"""
@author Dr. M. Nedim ALPDEMİR
"""
import numpy as np
from gekko import GEKKO
#import random
from Scene3DManager import CraneObj3D

class CraneController():
    def __init__(self, scene, isVisible=True): 
        self.m1 = 20 # mass of the cart
        self.B = np.array([[0],
                     [1],
                     [0],
                     [-1]])
        self.C = np.identity(4)
        self.reverse_op=False
        self.visible = isVisible
        self.crane3D = CraneObj3D(scene)
        self.oper_range = self.crane3D.cart_op_range
        self.t = []
        self.y = []
        self.dt = 0.01
        self.currTimeIdx = 0
        self.endTimeIdx = 0.0

    def getPickPoint(self):
        return self.crane3D.pend_end_pos

    def step(self, obj):
        self.currTimeIdx += 1
        if self.currTimeIdx >= self.endTimeIdx:
            finished = True    
            self.currTimeIdx = 0
        else:
            finished = False
            theta_i = self.y[2][self.currTimeIdx] 
            d_theta = self.y[2][self.currTimeIdx + 1] - self.y[2][self.currTimeIdx] 
            vel_i = self.y[1][self.currTimeIdx] 
            self.crane3D.stepMoveObject(self.dt, theta_i, d_theta, vel_i, obj.cube)

        return finished

    def SolveMoveObject(self, mass): 
        
        self.m2 = mass
        self.eps = self.m2/(self.m1 + self.m2)
        self.A = np.array([[0,1,0,0],
                     [0,0,self.eps,0],
                     [0,0,0,1],
                     [0,0,-1,0]])

        m = GEKKO(remote=False)
        x,y,u = m.state_space(self.A,self.B,self.C,D=None)
        m.time = np.linspace(0,8,101)
        fn = [0 if m.time[i]<6.2 else 1 for i in range(101)]      
        final = m.Param(value=fn)
        u[0].status = 1       
        m.Obj(final * ((y[0]+self.oper_range[0].x/4.0)**2 + y[1]**2 + y[2]**2 + y[3]**2))  
                      
        m.options.IMODE = 6
        m.solve()
        # returns time axis and y axes as a vector of arrays. y[0]=position(y), y[1]=velocity(v), 
        # y[2]=angle(theta), y[3]=angularVelocity(q)  and y[4]=forceInput(u)
        self.t = m.time
        self.y = y
        self.dt = self.t[1] - self.t[0]
        self.currTime = 0.0
        self.currTimeIdx = 0
        self.endTimeIdx = len(self.t) - 1
        return m.time, y 

    def pickObject(self, item3D):
        self.crane3D.pickItem(item3D.cube)
        finished = False

    def reset(self):
        self.crane3D.moveBack(2.0)

In [ ]:
"""
@author Dr. M. Nedim ALPDEMİR
"""
from pypdevs.DEVS import AtomicDEVS
#from CraneController import CraneController
#import random
import collections
from vpython import *
from Resource import Resource
from Scene3DManager import ConveyorBelt3D
from Scene3DManager import ConveyorType
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)
logging.getLogger().setLevel('DEBUG')

class State():
    def __init__(self):
        self.remaining = float("inf")
        self.current_time = 0.0
        self.out_item = None
        self.current_queue_size = 0.0
        # following arrays are for plots after the simulation
        self.time = []
        self.q_size_vals = []
        self.serv_util_level = []

class ShopFloorProcessor(AtomicDEVS):
    """
    
    """
    def __init__(self, scene, max_mass):
        AtomicDEVS.__init__(self, "ShopFloorProcessor")
        self.crane = Resource(num_servers = 1)
        self.scene = scene
        self.in_items = self.addInPort("in_items")
        self.out_moved_items = self.addOutPort("out_moved_items")
        self.dt = 0.1
        self.c = 0.05
        self.state = State()  
        self.max_WB_queue_size = 0
        self.currentIntTime = 0.0
        self.movingObj = None
        self.craneController = CraneController(scene.getRootScene())
        t_0 = 0.0      
        l = 12.0
        w = 1.2
        h = 1.0
        ol = l + 2
        y = self.craneController.getPickPoint().y - (max_mass / 20 + h / 2.0)

        inConvCent = vector(self.craneController.oper_range[0].x - l/2.0, y, self.craneController.oper_range[0].z)
        self.inConveyor3D = ConveyorBelt3D(scene.getRootScene(), inConvCent, vis_slots = 4, l=l, h=h,w=w)
        outConvCent = vector(self.craneController.oper_range[1].x + ol/2.0, y, self.craneController.oper_range[0].z)
        self.outConveyor3D = ConveyorBelt3D(scene.getRootScene(), outConvCent, vis_slots = 3, l=ol, h=h,w=w, type=ConveyorType.OUT)

    def intTransition(self):
        self.state.out_item = None  
        self.state.current_time += self.timeAdvance()
        if self.crane.getActiveServersNum() > 0:           
            finished = self.craneController.step(self.movingObj)
            logging.debug("crane allocated, stepping ..")
            if (finished):
                logging.debug("crane finished, releasing obj id : %d"%self.movingObj.id)
                logging.debug("resource : " + str(self.crane.servers))
                logging.debug("que : " + str(self.crane.queue))
                item_tuple = self.crane.release(self.movingObj.id, self.state.current_time)
                
                self.outConveyor3D.addItem(self.movingObj)
                
                self.craneController.reset()
                self.state.out_item = item_tuple
                if len(self.crane.getServers()) > 0:
                    scheduled_item = list(self.crane.getServers().values())[0]
                    self.craneController.SolveMoveObject(scheduled_item[0].mass)
                    self.movingObj = self.inConveyor3D.getItem()  
                    
                    logging.debug("seized crane for obj id : %d"%self.movingObj.id)
                    logging.debug("resource : " + str(self.crane.servers))
                    logging.debug("que : " + str(self.crane.queue))
                    if (self.movingObj != None):
                        self.craneController.pickObject(self.movingObj)
                        self.outConveyor3D.moveBelt()
                        if self.crane.getCurrentQueueSize() > 1:
                            self.inConveyor3D.moveBelt()
                        
                        

        self.state.current_queue_size = self.crane.getCurrentQueueSize()
        self.state.time.append(self.state.current_time)
        self.state.q_size_vals.append(self.state.current_queue_size)
        self.state.serv_util_level.append(self.crane.getActiveServersNum())
        return self.state

    def extTransition(self, inputs):
        self.state.current_time += self.elapsed
        try:
            if self.in_items in inputs:
                item = inputs[self.in_items]
                obj3D = self.scene.createMovingObj(item)
                self.inConveyor3D.addItem(obj3D)
                logging.debug("item arrived id : %d"%item.id)
                success = self.crane.seize(item, item.id, self.state.current_time)
                if success:
                    # crane was idle so start operation
                    # first solve the equations for this new mass
                    self.craneController.SolveMoveObject(item.mass)
                    self.movingObj = self.inConveyor3D.getItem()    
                    logging.debug("seized crane for obj id : %d"%self.movingObj.id)
                    logging.debug("resource : " + str(self.crane.servers))
                    logging.debug("que : " + str(self.crane.queue))
                    if (self.movingObj != None):
                        self.craneController.pickObject(self.movingObj)
                        self.outConveyor3D.moveBelt()

            self.max_WB_queue_size = max(self.max_WB_queue_size, self.crane.getCurrentQueueSize())
        except:
            logging.error("Exception when handling the incoming item with ID: %d "%item.id)
            raise 
        return self.state

    def timeAdvance(self):
        # Return remaining time; infinity when furnace is empty (wait for ingots to arrive from the port)
        if self.crane.getActiveServersNum == 0 and self.crane.getCurrentQueueSize == 0:
            self.state.remaining = float('inf')
        else:
            self.state.remaining = self.craneController.dt

        return self.state.remaining

    def outputFnc(self):
        # Output the new event on the output port
        return {self.out_moved_items: self.state.out_item}


In [ ]:
"""
@author Dr. M. Nedim ALPDEMİR
"""
from pypdevs.DEVS import AtomicDEVS

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

# Define the state of the collector as a structured object
class TransducerState(object):
    def __init__(self):
        # Contains received events and simulation time
        self.events = []
        self.current_time = 0.0
        # stats for plots
        self.time = []
        self.avg_q_wait = []
        self.avg_h_time = []
class SummaryStatistics(object):
    def __init__(self):
        # for furnace utilization we use ρ = λ / c*µ where, ρ is server utilization, 
        # λ is interarrival rate of items, c is number of identical servers
        # and µ is service rate 
        self.avg_queue_wait_time = 0.0
        self.server_utilization = 0
        self.avg_item_process_time = 0 # this is mean service time, i.e. inverse of µ
        self.avg_item_creation_interval = 0 # this mean interarrival time, i.e. inverse of λ

class Collector(AtomicDEVS):
    def __init__(self):
        AtomicDEVS.__init__(self, "Collector")
        self.state = TransducerState()
        self.sum_stats = SummaryStatistics()
        self.in_event = self.addInPort("in_event")
        self.total_process_time = 0
        self.total_item_creation_interval = 0
        self.total_queue_wait_time = 0
        self.total_num_items = 0

    def extTransition(self, inputs):
        # Update simulation time
        self.state.current_time += self.elapsed
        
        # Calculate time in queue
        event = inputs[self.in_event]
        if (event != None):
            logging.debug("TRANS: current time = %f5.2"%self.state.current_time)
           
            self.total_num_items += 1
            item_queue_wait_time = event[1].queue_end_time - event[1].queue_start_time
            item_heat_time = event[1].server_end_time  - event[1].server_start_time
            self.total_queue_wait_time += item_queue_wait_time
            self.total_process_time += item_heat_time
            self.total_item_creation_interval += event[0].creationTimeInterval
            self.sum_stats.avg_item_process_time = self.total_process_time  / self.total_num_items
            self.sum_stats.avg_queue_wait_time = self.total_queue_wait_time / self.total_num_items
            self.sum_stats.avg_item_creation_interval = self.total_item_creation_interval / self.total_num_items
        # add plot data
        self.state.time.append(self.state.current_time)
        self.state.avg_h_time.append(self.sum_stats.avg_item_process_time)
        self.state.avg_q_wait.append(self.sum_stats.avg_queue_wait_time)
        return self.state

    # Don't define anything else, as we only store events.
    # Collector has no behaviour of its own.


In [ ]:
"""
@author Dr. M. Nedim ALPDEMİR
"""
from pypdevs.DEVS import CoupledDEVS
#from ItemGenerator import ItemGenerator
#from ShopFloorProcessor import ShopFloorProcessor
#from Collector import Collector

class Workshop(CoupledDEVS):
    def __init__(self, item_arr_rate, min_mass, max_mass, scene):
        CoupledDEVS.__init__(self, "Workshop")
        # Define all atomic submodels 
        self.generator = self.addSubModel(ItemGenerator(item_arr_rate, min_mass, max_mass))
        self.processor = self.addSubModel(ShopFloorProcessor(scene, max_mass))
        self.transducer = self.addSubModel(Collector())
        # connect ports
        self.connectPorts(self.generator.item_arrival, self.processor.in_items)
        self.connectPorts(self.processor.out_moved_items, self.transducer.in_event)


In [ ]:
"""
@author Dr. M. Nedim ALPDEMİR
"""
from pypdevs.simulator import Simulator
#import random
import matplotlib.pyplot as plt
import logging
from Scene3DManager import Scene3D
#from Workshop import Workshop
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)
logging.getLogger().setLevel('INFO')
# Configuration:
max_sim_time = 300.0 # sec
mean_item_arr_interv = 5 # sec
mass_range = [5, 17]
scene = Scene3D()
rnd.seed(30)
model = Workshop(1 / mean_item_arr_interv, mass_range[0], mass_range[1], scene) 
# PythonPDEVS specific setup and configuration
sim = Simulator(model)
sim.setClassicDEVS()
sim.setTerminationTime(max_sim_time)
sim.simulate()

# Gather statistical information for output
max_q_size = model.processor.max_WB_queue_size  
curr_q_size = model.processor.state.current_queue_size
avg_gen_arrival_dt = model.generator.state.avg_delta_arival 
gen_item_num = model.generator.state.gen_num
processed_item_num = model.transducer.total_num_items
avg_item_wait_time = model.transducer.sum_stats.avg_queue_wait_time
avg_service_time = model.transducer.sum_stats.avg_item_process_time
avg_exp_interarriv_time = model.transducer.sum_stats.avg_item_creation_interval
service_utilization = avg_service_time / (avg_exp_interarriv_time * 1)

logging.info("  --------------------------------------------------------------------")
logging.info("  SUMMARY RESULTS")
logging.info("  --------------------------------------------------------------------")
logging.info("  {:25s} : {:10.3f}".format("total generated items", gen_item_num))
logging.info("  {:25s} : {:10.3f}".format("total processed items", processed_item_num))
logging.info("  {:25s} : {:10.3f}".format("avg item wait time", avg_item_wait_time))
logging.info("  {:25s} : {:10.3f}".format("avg item interarv time", avg_gen_arrival_dt))
logging.info("  {:25s} : {:10.3f}".format("avg service time", avg_service_time))
logging.info("  {:25s} : {:10.3f}".format("service utilization", service_utilization))
logging.info("  {:25s} : {:10.3f}".format("maximum WB queue size", max_q_size))
logging.info("  {:25s} : {:10.3f}".format("current WB queue size", curr_q_size))

